# Сцена 1

![](Scene1.png)

## эффекты: 
####    отбрасывание тени на другие объекты(1 балл)
####    зеркальные грани(1 балл)
####    объемный источник освещения(3 балла)
####    нечеткие тени(3 балла)
####    нечеткие отражения(4 балла)

### итого за первую сцену 12 баллов

In [11]:
from PIL import Image 
import math
from tqdm import tqdm

##### Размер окна наблюдателя

In [12]:
d = 10
Vw = 10
Vh = 10

In [113]:
#sphere
center = [ (-2, -2, 10), (-3, 3, 15), (10, 7, 20), (1.5, 3, 10), (2.2, 0, 5), (1, 0, 7), (0.7, -3, 7)]
clr = [ (0, 120, 250), (20, 210, 255), (120, 20, 120), (121, 167, 141), (0, 129, 100), (255,255,255),(229, 139, 173)]
radius = [3, 1, 10, 1, 1, 0.09, 1]
specular = [ 60, 40, 40, 50, 30, 20, 50]
reflective = (0.6, 0.4, 0.4, 0.4, 0.5, 0.4, 0.8)
t_max = 1

#light
intensity = (0.2, 0.8)
light_position = (0.7, 0, 7)

In [66]:
def tovec(A, B):
    return (B[0]-A[0], B[1]-A[1], B[2]-A[2])

def CanvasToViewport(x, y): 
    return (x*Vw/Cw, -y*Vh/Ch, d)

def length(V):
    return math.sqrt(V[0]*V[0] + V[1]*V[1] + V[2]*V[2])

def dot(V1, V2):
    return V1[0]*V2[0] + V1[1]*V2[1] + V1[2]*V2[2]

def neg(V):
    return (-V[0], -V[1], -V[2])

def sumv(V1, V2):
    return (V1[0]+V2[0], V1[1]+V2[1], V1[2]+V2[2])

def divk(V, k):
    return (V[0]*k, V[1]*k, V[2]*k)

def divv(V, a):
    return (V[0]/a, V[1]/a, V[2]/a)

def ReflectRay(R, N):
    X = (2*N[0]*dot(N,R)-R[0], 2*N[1]*dot(N,R)-R[1], 2*N[2]*dot(N,R)-R[2])
    return X

In [67]:
def IntersectRaySphere(O, D, i):
    C = center[i]
    r = radius[i]
    
    OC = (O[0] - C[0], O[1] - C[1], O[2] - C[2])

    k1 = dot(D, D)
    k2 = 2 * dot(OC, D)
    k3 = dot(OC, OC) - r*r

    discriminant = k2*k2 - 4*k1*k3
    if (discriminant < 0):
        return (math.inf, math.inf) 

    t1 = (-k2 + math.sqrt(discriminant)) / (2*k1)
    t2 = (-k2 - math.sqrt(discriminant)) / (2*k1)
    return (t1, t2)

In [68]:
def ComputeLighting(P, N, V, s):
    i = 0.0
    for num in range(0, 2):
        if (num == 0):
            i += intensity[0]
        else:    
            L = (light_position[0] - P[0], 
                 light_position[1] - P[1], 
                 light_position[2] - P[2])

            v = []
            
            tmp = (light_position[0] - P[0]+0.1, light_position[1] - P[1], light_position[2] - P[2])
            v.append(tmp)
            tmp = (light_position[0] - P[0], light_position[1] - P[1]+0.1, light_position[2] - P[2])
            v.append(tmp)
            tmp = (light_position[0] - P[0], light_position[1] - P[1], light_position[2] - P[2]+0.1)
            v.append(tmp)
            tmp = (light_position[0] - P[0]-0.1, light_position[1] - P[1], light_position[2] - P[2])
            v.append(tmp)
            tmp = (light_position[0] - P[0], light_position[1] - P[1]-0.1, light_position[2] - P[2])
            v.append(tmp)
            tmp = (light_position[0] - P[0], light_position[1] - P[1], light_position[2] - P[2]-0.1)
            v.append(tmp)    
                
            for k in range(0, 6):
                shadow_sphere = ClosestIntersection(P, v[k], 0.001, t_max)[0]
                shadow_t = ClosestIntersection(P, v[k], 0.001, t_max)[1]
            
                if (shadow_sphere != -1 and shadow_sphere != 5):
                    continue
                intens = intensity[1] / 4
                n_dot_l = dot(N, L)
        
                if (n_dot_l > 0):
                    i += intens * n_dot_l / (length(N)*length(L)) 
    
                if (s != -1): 
                    R = (2*N[0]*n_dot_l - L[0], 2*N[1]*n_dot_l - L[1], 2*N[2]*n_dot_l - L[2])
                    r_dot_v = dot(R, V)
                    if (r_dot_v > 0):
                        i += intens*pow(r_dot_v/(length(R)*length(V)), s)
    return i

In [104]:
def ClosestIntersection(O, D, t_min, t_max):
    closest_t = 1000000
    closest_sphere = -1
    tmp = closest_t
    for k in range(0, 7):
        
        t1 = IntersectRaySphere(O, D, k)[0]
        t2 = IntersectRaySphere(O, D, k)[1]
        
        if (t1 > t_min and t1 < t_max and t1 < closest_t):
            tmp = closest_sphere
            closest_t = t1
            closest_sphere = k
        
        if (t2 > t_min and t2 < t_max and t2 < closest_t):
            tmp = closest_t
            closest_t = t2
            closest_sphere = k 
        
    return (closest_sphere, closest_t, tmp)

In [105]:
def TraceRay(O, D, t_min, t_max, depth): 
    
    closest_sphere = ClosestIntersection(O, D, t_min, t_max)[0]
    closest_t = ClosestIntersection(O, D, t_min, t_max)[1]
    tmp = ClosestIntersection(O, D, t_min, t_max)[2]
    
    if (closest_sphere == -1):
        return (50, 50, 50) 
       
    if (closest_sphere == 5):
        return(255, 255, 255)
    
    P = (O[0] + closest_t*D[0], O[1] + closest_t*D[1], O[2] + closest_t*D[2])      
    X = (P[0] - center[closest_sphere][0], 
         P[1] - center[closest_sphere][1], 
         P[2] - center[closest_sphere][2] )
    lenN = length(X)
    N = (X[0] / lenN, X[1] / lenN, X[2] / lenN) 
    
    X = (-D[0], -D[1], -D[2])
        
    local_color = (clr[closest_sphere][0]*ComputeLighting(P, N, X, specular[closest_sphere]), 
                   clr[closest_sphere][1]*ComputeLighting(P, N, X, specular[closest_sphere]), 
                   clr[closest_sphere][2]*ComputeLighting(P, N, X, specular[closest_sphere]) )
    r = reflective[closest_sphere]
    if (depth <= 0 or r <= 0):
        return local_color
    
 
    R = ReflectRay(X, N)
    V = tovec(P, R)

    ref = []
    c = []
    
    R1 = (R[0]+0.5, R[1], R[2])
    V1 = tovec(P, R1)
    cos = dot(V, V1)/length(V)*length(V1)
    ref.append(R1)
    c.append(cos)
    
    R1 = (R[0], R[1]+0.5, R[2])
    V1 = tovec(P, R1)
    cos = dot(V, V1)/length(V)*length(V1)
    ref.append(R1)
    c.append(cos)
    
    R1 = (R[0], R[1], R[2]+0.5)
    V1 = tovec(P, R1)
    cos = dot(V, V1)/length(V)*length(V1)
    ref.append(R1)
    c.append(cos)
    
    R1 = (R[0]-0.5, R[1], R[2])
    V1 = tovec(P, R1)
    cos = dot(V, V1)/length(V)*length(V1)
    ref.append(R1)
    c.append(cos)
    
    R1 = (R[0], R[1]-0.5, R[2])
    V1 = tovec(P, R1)
    cos = dot(V, V1)/length(V)*length(V1)
    ref.append(R1)
    c.append(cos)
    
    R1 = (R[0], R[1], R[2]-0.5)
    V1 = tovec(P, R1)
    cos = dot(V, V1)/length(V)*length(V1)
    ref.append(R1)
    c.append(cos)
    
    R1 = (R[0]+0.5, R[1]-0.5, R[2])
    V1 = tovec(P, R1)
    cos = dot(V, V1)/length(V)*length(V1)
    ref.append(R1)
    c.append(cos)
    
    R1 = (R[0]-0.5, R[1], R[2]-0.5)
    V1 = tovec(P, R1)
    cos = dot(V, V1)/length(V)*length(V1)
    ref.append(R1)
    c.append(cos)
    
    s = (0, 0, 0)
    
    for k in range(0, 8):
        co = TraceRay(P, ref[k], 0.001, math.inf, depth - 1)
        co = divk(co, c[k])
        s = sumv(s, co)
    
    reflected_color = divv(s, sum(c))    
    #reflected_color = TraceRay(P, R, 0.001, math.inf, depth - 1)

    
    res = (local_color[0]*(1 - r) + reflected_color[0]*r,
           local_color[1]*(1 - r) + reflected_color[1]*r,
           local_color[2]*(1 - r) + reflected_color[2]*r)
    return res

In [114]:
image = Image.open(r'canvas.png')  
  
Cw, Ch = image.size 
O = (0, 0, 0)  

for x in tqdm(range(-300, 300)):
    x1 = x + 300
    for y in range(-300, 300):
        y1 = y + 300
        color = TraceRay(O, CanvasToViewport(x, y), 0, math.inf, 1)
        image.putpixel((x1, y1), tuple(int(c) for c in color) )  
image.show() 

100%|██████████| 600/600 [12:51<00:00,  1.29s/it]
